In [1]:
# STANDARD LIBRARY IMPORTS
import re
import csv
import os
import datetime
import subprocess
import sys # for args

# THIRD PARTY IMPORTS
import praw

# PROJECT IMPORTS
from setup import reddit
from settings import *
from tools import *
from inbox import check_messages
from reminders import send_reminders
from log_missed_comment import manually_award_comment

subreddit = reddit.subreddit(TARGET_SUBREDDIT)

Loaded 13 users who have opted out from follow-ups.
Loaded 13 users who have opted out from reminders.


Version 7.5.0 of praw is outdated. Version 7.7.1 was released Tuesday July 11, 2023.


# Step 1: Get the last awarded post

In [2]:
# get the last awarded post. We don't need to look back past this time.
last_award = next( reddit.user.me().comments.new() )

In [3]:
print( last_award.permalink )

/r/photocritique/comments/1bpmfu2/my_first_ever_picture_with_a_professional_camera/kwxemtg/


# Step 2: Search the most recent 1,000 comments, and record their IDs

In [4]:
num_processed_comments = 0
num_awarded_comments   = 0

# we can only go back 1,000 comments :|
for awarding_comment in subreddit.comments(limit=None):
    num_processed_comments += 1
    
    if (awarding_comment.author is not None and awarding_comment.author.name != 'AutoModerator') and comment_contains_token( awarding_comment ):
        print(f'Should we award points based on comment {awarding_comment.id} by u/{awarding_comment.author.name}?')
        num_awarded_comments += 1

    if awarding_comment.created_utc <= last_award.created_utc:
        # we've gone back far enough
        print('Yay! There were less than 1,000 comments since the bot crashed.\n')
        break
        
print(f'\n\nSearched {num_processed_comments} comments. Found {num_awarded_comments} that might need awarding.')
        
print(f'There is a gap of {( awarding_comment.created_utc - last_award.created_utc ) / 60 / 60:.1f} hours between the last comment we searched, and the last award.')
print('Hopefully the above number is 0 or small!')

Should we award points based on comment kxwxzu2 by u/Boring_Peanut_4369?
Should we award points based on comment kxwxovt by u/Boring_Peanut_4369?
Should we award points based on comment kxtytby by u/GazNPhoto?
Should we award points based on comment kxtwokx by u/SiMoon_MD?
Should we award points based on comment kxsoihd by u/AdventureJuntos?
Should we award points based on comment kxrmp1z by u/FaxCelestis?
Should we award points based on comment kxqjwvx by u/MyRoadTaken?
Should we award points based on comment kxptqk0 by u/MyRoadTaken?
Should we award points based on comment kxpnw1f by u/MyRoadTaken?
Should we award points based on comment kxpm1kn by u/MyRoadTaken?
Should we award points based on comment kxplyge by u/MyRoadTaken?
Should we award points based on comment kxplwlw by u/MyRoadTaken?
Should we award points based on comment kxp3hio by u/auraria?
Should we award points based on comment kxookga by u/lonerockz?
Should we award points based on comment kxoo3aa by u/ExpatInFarEast?

# Step 3: Manually award all the comments above.

Reddit will rate limit you at some point, so you'll need to wait a few mins if you have more than ~15 comments to award.

To award comments, copy the ID above into the below string, and run the cell. Repeat for all comments listed above. You may want to manually check things like:

- don't award to top level comments? 

Are these things still in `run.py`?

In [5]:
from time import sleep

In [10]:
missing_awards = [
# 'kxwxzu2',
# 'kxwxovt',
# 'kxtytby',
# 'kxtwokx',
# 'kxsoihd',
# 'kxrmp1z',
# 'kxqjwvx',
# 'kxptqk0',
# 'kxpnw1f',
# 'kxpm1kn',
    #
# 'kxplyge',
# 'kxplwlw',
# 'kxp3hio',
# 'kxookga',
# 'kxoo3aa',
# 'kxnvuot',
# 'kxnb13i',
# 'kxl3oi4',
# 'kxl3m5w',
# 'kxjhgd3',
    #
'kxjgbed',
'kxjfrmm',
'kxil8tq',
'kxibc7z',
'kxia8c1',
'kxi8j2d',
]

In [11]:
for award_id in missing_awards:
    manually_award_comment(award_id)
    sleep(10)

Manually awarding points based on comment kxjgbed by u/jr_photography.
Manually awarding points based on comment kxjfrmm by u/jr_photography.
Manually awarding points based on comment kxil8tq by u/ArgeberazziSostrepi.
Manually awarding points based on comment kxibc7z by u/vyralinfection.
Manually awarding points based on comment kxia8c1 by u/vyralinfection.
Manually awarding points based on comment kxi8j2d by u/Ashexy-.


# Step 5: Manually look for missed awards that need checking

I would do this by starting at /new, scrolling back to post from the below last-checked comment, and then manually opening all the posts after that and searching for `!CritiquePoint`. Do this until you are fairly certain you've gone far enough.

In [15]:
print('This is the last comment we were able to check:')
print(awarding_comment.permalink)

print('\nComments older than this may need checking manually.')

This is the last comment we were able to check:
/r/photocritique/comments/xza4r1/feedback_request_on_portrait/irmugrd/

Comments older than this may need checking manually.


In [42]:
#  manually_award_comment('irah590')

Manually awarding points based on comment irah590 by u/Koosen-.


In [ ]:
# https://old.reddit.com/r/photocritique/comments/xwqlb5/my_thought_was_better_than_the_execution/irah590/